# **2. A neurális hálózatok matematikai építőkövei**

Ez a fejezet ezekkel foglalkozik:
* Első példa a neurális hálózatra
* Tenzorok és tenzorműveletek
* Hogyan tanulnak a neurális hálózatok a visszaszaporítás és a gradiens süllyedés útján

A mély tanulás megértéséhez sok egyszerű matematikai fogalom ismerete szükséges: tenzorok, tenzorműveletek, differenciálás, gradiens süllyedés stb. Ebben a fejezetben az lesz a célunk, hogy kiépítsük az intuíciót ezekkel a fogalmakkal kapcsolatban anélkül, hogy túlzottan technikai jellegűvé válnánk. Különösen kerüljük a matematikai jelölést, amely szükségtelen akadályokat állíthat fel azok számára, akik nem rendelkeznek matematikai háttérrel, és nem szükséges jól elmagyarázni a dolgokat. A matematikai műveletek legpontosabb, legegyértelműbb leírása a végrehajtható kód.
Ahhoz, hogy elegendő kontextust biztosítsunk a tenzorok és a gradiens süllyedés bemutatásához, a fejezetet egy neurális hálózat gyakorlati példájával kezdjük. Ezután minden bevezetett új koncepciót pontról pontra átnézünk. Ne feledje, hogy ezek a fogalmak nélkülözhetetlenek lesznek a következő fejezetekben található gyakorlati példák megértéséhez!

{27.o:}
Miután elolvasta ezt a fejezetet, intuitív módon megérti a mély tanulás mögött meghúzódó matematikai elméletet, és készen áll arra, hogy a 3. fejezetben belemerüljön a Keras és a TensorFlow témakörbe.

## 2.1. Első pillantás egy neurális hálózatra
Nézzünk egy konkrét példát egy neurális hálózatra, amely a Python Keras könyvtárat használja a kézzel írt számjegyek osztályozásának megtanulására. Hacsak nincs már tapasztalata a Keras vagy hasonló könyvtárakkal kapcsolatban, nem fog azonnal mindent megérteni erről az első példáról. Rendben van. A következő fejezetben a példa minden elemét áttekintjük, és részletesen elmagyarázzuk. Tehát ne aggódjon, ha egyes lépések önkényesnek vagy varázslatosnak tűnnek számodra! Valahol el kell kezdenünk.
A probléma, amit itt próbálunk megoldani, az, hogy a kézzel írt számjegyek (28 × 28 pixeles) szürkeárnyalatos képeit a 10 kategóriájukba (0-tól 9-ig) soroljuk. A gépi tanulási közösség klasszikusának számító MNIST adatkészletet fogjuk használni, amely majdnem olyan régóta létezik, mint maga a terület, és intenzíven tanulmányozták. Ez egy 60 000 képzési képből és 10 000 tesztképből álló készlet, amelyet a National Institute of Standards and Technology (a NIST az MNIST-ben) állított össze az 1980-as években. Az MNIST „megoldását” a mély tanulás „Hello Worldjének” tekintheti – ezzel ellenőrizheti, hogy az algoritmusok a várt módon működnek-e. Ahogy gépi tanulással foglalkozó gyakornok leszel, az MNIST újra és újra megjelenik a tudományos közleményekben, blogbejegyzésekben stb. Néhány MNIST mintát láthat a 2.1. ábrán.

<img src="figs/f2.1_.jpg">

**2.1. ábra:** MNIST minta számjegyek

MEGJEGYZÉS

> A gépi tanulásban egy osztályozási probléma kategóriáját osztálynak nevezzük. Az adatpontokat mintáknak nevezzük. Az adott mintához tartozó osztályt címkének nevezzük.

Ne akarja most megpróbálni reprodukálni ezt a példát a gépén. Ha szeretné, először be kell állítania egy mély tanulási munkaterületet, amelyről a 3. fejezetben olvashat.
Az MNIST-adatkészlet a Kerasba előre telepítve, négy NumPy-tömbből álló készlet formájában érkezik.

### 2.1. lista: Az MNIST adatkészlet betöltése Kerasba

In [ ]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

`train_images` és `train_labels` alkotják a betanítási halmazt, azokat az adatokat, amelyekből a modell tanulni fog. A modell ezután tesztelésre kerül a `test_images` és a `test_labels` tesztkészleten.
{28.o:}
A képek NumPy tömbökként vannak kódolva, a címkék pedig számjegyek 0-tól 9-ig terjedő tömbjei. A képek és a címkék egy az egyben megfelelnek.

Nézzük az edzési adatokat:
```
>>> train_images.shape
(60000, 28, 28)
>>> len(train_labels)
60000
>>> train_labels
array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)
```

És itt vannak a tesztadatok:
```
>>> test_images.shape
(10000, 28, 28)
>>> len(test_labels)
10000
>>> test_labels
array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)
```

A munkafolyamat a következő lesz: Először is betápláljuk a neurális hálózatba a `train_images` és `train_labels`betanítási adatokat. A hálózat ezután megtanulja társítani a képeket és a címkéket. Végül megkérjük a hálózatot, hogy készítsen előrejelzéseket a `test_images` számára, és ellenőrizzük, hogy ezek az előrejelzések egyeznek-e a `test_labels` címkéivel.

Építsük fel a hálózatot – ne feledje, hogy még nem várható el, hogy mindent megértsen ebből a példából.

### 2.2. lista: A hálózati architektúra

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

A neurális hálózatok alapvető építőköve a réteg. A réteget az adatok szűrőjének tekintheti: bizonyos adatok bekerülnek, és hasznosabb formában kerülnek ki. Pontosabban, a rétegek *reprezentációkat* vonnak ki a beléjük betáplált adatokból – remélhetőleg olyan reprezentációkat, amelyek értelmesebbek az adott probléma szempontjából. A mély tanulás nagy része egyszerű rétegek összeláncolásából áll, amelyek a progresszív *adatdesztilláció* egy formáját valósítják meg. A mélytanulási modell olyan, mint az adatfeldolgozás szitája, amely egyre finomabb adatszűrőkből – a rétegekből – áll.

Itt a modellünk két sűrű réteg sorozatából áll, amelyek sűrűn összefüggő (más néven *teljesen összefüggő*) idegi rétegek. A második (és utolsó) réteg egy 10-utas *softmax osztályozási* réteg, ami azt jelenti, hogy 10 valószínűségi pontszámból álló tömböt fog visszaadni (összeadva 1-et). Minden pontszám annak a valószínűsége lesz, hogy az aktuális számjegyű kép a 10 számjegyű osztályunk valamelyikébe tartozik.
{29.o:}

Ahhoz, hogy a modell edzésre kész legyen, további három dolgot kell kiválasztanunk az *összeállítási* lépés részeként:
* *Egy optimalizálót* – Az a mechanizmus, amelyen keresztül a modell frissíti magát a látott betanítási adatok alapján, hogy javítsa a teljesítményét.
* *Egy veszteségfüggvényt* – Hogyan tudja a modell mérni a teljesítményét a betanítási adatokon, és így hogyan tudja majd a megfelelő irányba terelni magát.
* *A képzés és a tesztelés során figyelendő mutatókat* – Itt csak a pontossággal (a helyesen besorolt képek töredékével) foglalkozunk.
A veszteségfüggvény és az optimalizáló pontos célja a következő két fejezetben világossá válik.

### 2.3. lista: Az összeállítás lépése

In [ ]:
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

A képzés előtt az adatokat a modell által elvárt alakra alakítjuk át, és úgy méretezzük, hogy minden érték a `[0, 1]` intervallumban legyen. Korábban a képzési képeinket egy `uint8` típusú alaktömbben `(60000, 28, 28)` tároltuk `[0, 255]` intervallumban. Átalakítjuk egy `float32` alakú tömbbe `(60000, 28 * 28)`, 0 és 1 közötti értékekkel.

### 2.4. lista: A képadatok előkészítése

In [ ]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

Most már készen állunk a modell betanítására, ami a Kerasban a modell `fit()` metódusának meghívásával történik – most *illesztjük* a modellt a betanítási adataihoz.

### 2.5. lista: A modell „illesztése”


In [ ]:
>>> model.fit(train_images, train_labels, epochs=5, batch_size=128)
Epoch 1/5
60000/60000 [===========================] - 5s - loss: 0.2524 - acc: 0.9273
Epoch 2/5
51328/60000 [=====================>.....] - ETA: 1s - loss: 0.1035 - acc: 0.9692

A betanítás során két mennyiség jelenik meg: a modell veszttesége a betanítási adatok felett, és a modell pontossága a betanítási adatok felett. Az edzésadatokon gyorsan elérjük a 0,989-es (98,9%-os) pontosságot.

Most, hogy van egy betanított modellünk, felhasználhatjuk új számjegyek osztályvalószínűségének előrejelzésére – olyan képekre, amelyek nem képezték a betanítási adatok részét, például a tesztkészletből. {30.o:}

### 2.6. lista: A modell használata előrejelzések készítéséhez


In [ ]:
>>> test_digits = test_images[0:10]
>>> predictions = model.predict(test_digits)
>>> predictions[0]
array([1.0726176e-10, 1.6918376e-10, 6.1314843e-08, 8.4106023e-06,
       2.9967067e-11, 3.0331331e-09, 8.3651971e-14, 9.9999106e-01,
       2.6657624e-08, 3.8127661e-07], dtype=float32)

Ebben a tömbben az i indexek minden száma megfelel annak a valószínűségének, hogy a `test_digits[0]` számjegykép az i. osztályba tartozik.

Ennek az első tesztszámjegynek a legmagasabb a valószínűségi pontszáma (0,99999106, majdnem 1) a 7-es indexnél, tehát modellünk szerint 7-nek kell lennie:
```
>>> predictions[0].argmax()
7
>>> predictions[0][7]
0.99999106
```
Ellenőrizni tudjuk, hogy a tesztcímke megegyezik-e:

```
>>> test_labels[0]
7
```
Átlagosan mennyire jó a modellünk az ilyen soha nem látott számjegyek osztályozásában? Ellenőrizzük a teljes tesztkészlet átlagos pontosságának kiszámításával.

### 2.7. lista: A modell kiértékelése új adatokon

In [ ]:
>>> test_loss, test_acc = model.evaluate(test_images, test_labels)
>>> print(f"test_acc: {test_acc}")
test_acc: 0.9785

A tesztkészlet pontossága 97,8% - ez egy kicsit kisebb, mint az edzéskészlet pontossága (98,9%). A képzési pontosság és a tesztpontosság közötti különbség a *túlillesztés* példája: az a tény, hogy a gépi tanulási modellek általában rosszabbul teljesítenek az új adatokon, mint a betanítási adatokon. A túlillesztés a 3. fejezet központi témája.

Ezzel végére is értünk az első példánknak – most láttad, hogyan tudsz felépíteni és betanítani egy neurális hálózatot, hogy a kézzel írt számjegyeket osztályozza, és mindezt kevesebb, mint 15 Python-kódsorban. Ebben és a következő fejezetben részletesen kitérünk minden alkotórészre, amelyet most előre megnéztünk, és tisztázzuk, mi történik a színfalak mögött. Megismerheti a tenzorokat, a modellbe kerülő adattároló objektumokat; tenzorműveleteket, mely rétegekből készülnek; és a gradiens süllyedést, amely lehetővé teszi, hogy modellje tanuljon a képzési példákból. {31.o:}

## **2.2. Adatábrázolások neurális hálózatokhoz**

Az előző példában többdimenziós NumPy tömbökben, más néven *tenzorokban* tárolt adatokból indultunk ki. Általában minden jelenlegi gépi tanulási rendszer tenzorokat használ alapvető adatszerkezetként. A tenzorok alapvetőek a területen – annyira alapvetőek, hogy a TensorFlow-t róluk nevezték el. Tehát mi a tenzor?
A tenzor lényegében egy adattároló – mégpedig általában numerikus adatok. Tehát ez egy számtároló. Lehet, hogy már ismeri a mátrixokat, amelyek 2. rangú tenzorok: a tenzorok a mátrixok tetszőleges számú *dimenzióra* történő általánosítása (megjegyzendő, hogy a tenzorokkal összefüggésben egy dimenziót gyakran tengelynek neveznek).



### 2.2.1 Skalárok (nulladrendű tenzorok)

A csak egy számot tartalmazó tenzort *skalárnak* (vagy skalártenzornak, vagy nulladrendű tenzornak vagy 0D tenzornak) nevezzük. A NumPy-ban a `float32` vagy `float64` szám skalártenzor (vagy skalártömb). A NumPy tenzor tengelyeinek számát az `ndim` attribútummal tudjuk megjeleníteni; egy skaláris tenzornak 0 tengelye van (`ndim == 0`). A tenzor tengelyeinek számát *rendjének* is nevezik. Íme egy NumPy skalár:
```
>>> import numpy as np
>>> x = np.array(12)
>>> x
array(12)
>>> x.ndim
0
```


### 2.2.2 Vektorok (elsőrendű tenzorok)

A számokból álló tömböt *vektornak*, elsőrendű tenzornak vagy 1D tenzornak nevezzük. Egy elsőrendű tenzornak pontosan egy tengelye van. A következő egy NumPy vektor:
```
>>> x = np.array([12, 3, 6, 14, 7])
>>> x
array([12, 3, 6, 14, 7])
>>> x.ndim
1
```
Ennek a vektornak öt bejegyzése van, ezért *5-dimenziós vektornak* nevezik. Ne keverje össze az 5D vektort az 5D tenzorral! Egy 5D vektornak csak egy tengelye van, és a tengelye mentén öt dimenziója, míg az 5D tenzornak öt tengelye van (és tetszőleges számú dimenziója lehet minden tengely mentén). A *dimenzionalitás* jelölheti vagy egy adott tengely mentén lévő bejegyzések számát (mint az 5D vektorunk esetében), vagy egy tenzor tengelyeinek számát (például egy 5D tenzor), ami időnként zavaró lehet. Ez utóbbi esetben technikailag helyesebb *ötödrendű tenzorról* beszélni (a tenzor rangja a tengelyek száma), de ettől függetlenül gyakori az *5D tenzor* kétértelmű jelölés. {32.o:}

### 2.2.3 Mátrixok (másodrendű tenzorok)

A vektorok tömbje egy *mátrix*, vagy másodrendű tenzor vagy 2D tenzor. A mátrixnak két tengelye van (ezeket gyakran *sornak* és *oszlopnak* nevezik). A mátrixot vizuálisan értelmezheti téglalap alakú számrácsként. Ez egy NumPy mátrix:
```
>>> x = np.array([[5, 78, 2, 34, 0],
                  [6, 79, 3, 35, 1],
                  [7, 80, 4, 36, 2]])
>>> x.ndim
2
```
Az első tengely bejegyzéseit *soroknak*, a második tengely bejegyzéseit *oszlopoknak* nevezzük. Az előző példában `[5, 78, 2, 34, 0]` az x első sora, és az `[5, 6, 7]` az első oszlopa.

### 2.2.4. Harmad- és magasabb rendű tenzorok
Ha az ilyen mátrixokat egy új tömbbe csomagoljuk, akkor egy harmadrendű tenzort (vagy 3D tenzort) kapunk, amelyet vizuálisan számkockaként értelmezhetünk. A következő egy NumPy harmadrendű tenzor:
```
>>> x = np.array([[[5, 78, 2, 34, 0],
                   [6, 79, 3, 35, 1],
                   [7, 80, 4, 36, 2]],
                  [[5, 78, 2, 34, 0],
                   [6, 79, 3, 35, 1],
                   [7, 80, 4, 36, 2]],
                  [[5, 78, 2, 34, 0],
                   [6, 79, 3, 35, 1],
                   [7, 80, 4, 36, 2]]])
>>> x.ndim
3
```
Ha a harmadrendű tenzorokat egy tömbbe csomagolja, létrehozhat egy negyedrendű tenzort, és így tovább. A mélytanulás során általában a 0-tól 4-ig terjedő tenzorokat kell manipulálni, de ez akár 5-ig is felmehet, ha videoadatokat dolgoz fel.

### 2.2.5 Kulcs attribútumok

A tenzort három fő attribútum határozza meg:
* *Tengelyek száma (rend)* – Például egy harmadrendű tenzornak három tengelye van, a mátrixnak pedig két tengelye van. Ezt a Python-könyvtárak, például a NumPy vagy a TensorFlow a tenzor `ndim`-jének is nevezik.
* *Alakzat* – Ez egy egész szám, amely leírja, hogy a tenzornak hány dimenziója van az egyes tengelyek mentén. Például az előző mátrixpéldának van alakja `(3, 5)`, a harmadrendű tenzoros példának pedig az alakja `(3, 3, 5)`. A vektornak egyetlen elemű alakja van, például `(5,)`, míg a skalárnak az alakja üres, `()`. {33}
* *Adattípus (általában `dtype`-nak nevezik a Python könyvtárakban)* – Ez a tenzorban található adatok típusa; Például egy tenzor típusa lehet `float16, float32, float64, uint8` stb. A TensorFlow-ban valószínűleg találkozhatsz `sztring` tenzorokkal is.

Hogy ezt konkrétabbá tegyük, tekintsünk vissza az MNIST példában feldolgozott adatokra. Először betöltjük az MNIST adatkészletet:
```
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
```
Ezután megjelenítjük a `train_images` tenzor tengelyeinek számát, az `ndim` attribútumot:
```
>>> train_images.ndim
3
```
Íme az alakja:
```
>>> train_images.shape
(60000, 28, 28)
```
És az adattípusa, a `dtype` attribútum:
```
>>> train_images.dtype
uint8
```
Tehát itt van egy 8 bites egész számokból álló harmadrendű tenzor. Pontosabban, ez egy 60 000 darab 28 × 28 egész számból álló mátrixból felépített tömb. Mindegyik ilyen mátrix egy szürkeárnyalatos kép, 0 és 255 közötti értékű pixelekkel.

Jelenítsük meg a negyedik számjegyet ebben a harmadrendű tenzorban a Matplotlib könyvtár segítségével (ami egy jól ismert Python adatvizualizációs könyvtár, amely előre telepítve van a Colabban); lásd a 2.2 ábrát.

<img src="figs/f2.2_.jpg">

**2.2. ábra:** Adatkészletünk negyedik mintája

## -2.8. lista: A negyedik számjegy megjelenítése



In [ ]:
import matplotlib.pyplot as plt
digit = train_images[4]
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()

Természetesen a megfelelő címke a 9-es egész szám:
```
>>> train_labels[4]
9
```

### 2.2.6 Tenzorok manipulálása a NumPy-ban

Az előző példában kiválasztottunk egy adott számjegyet az első tengely mellett a `train_images[i]` szintaxis használatával. A tenzorban meghatározott elemek kiválasztását *tenzorszeletelésnek* nevezzük.
Nézzük meg a tenzorszeletelési műveleteket, amelyeket a NumPy tömbökön végezhet.
A következő példa a #10-től #100-ig terjedő számjegyeket választja ki (a #100 nem szerepel), és egy alakzattömbbe helyezi őket `(90, 28, 28)`:
```
>>> my_slice = train_images[10:100]
>>> my_slice.shape
(90, 28, 28)
```
Ez egyenértékű ezzel a részletesebb jelöléssel, amely minden tenzortengely mentén meghatároz egy kezdő- és egy végindexet a szelethez. Vegye figyelembe, hogy a `:` egyenértékű a teljes tengely kiválasztásával:
```
>>> my_slice = train_images[10:100, :, :]  <--- Egyenértékű az előző példával
>>> my_slice.shape
(90, 28, 28)
>>> my_slice = train_images[10:100, 0:28, 0:28] <---Szintén egyenértékű az  
                                                    előző példával
>>> my_slice.shape
(90, 28, 28)
```
Általában az egyes tenzortengelyek mentén tetszőleges két index közül választhat szeleteket. Ha például az összes kép jobb alsó sarkában 14 × 14 képpontot szeretne kiválasztani, tegye a következőket:
```
my_slice = train_images[:, 14:, 14:]
```
Lehetőség van negatív indexek használatára is. Hasonlóan a Python-listák negatív indexeihez, az aktuális tengely végéhez viszonyított pozíciót jelzik. Ha a képeket középen 14 × 14 képpontos foltokra szeretné levágni, tegye a következőket:
```
my_slice = train_images[:, 7:-7, 7:-7]
```

### 2.2.7 Az adatkötegek fogalma

{35.o:} Általánosságban elmondható, hogy az első tengely (0-s tengely, mert az indexelés 0-tól kezdődik) a mélytanulás során használt összes adattenzorban a *mintatengely* lesz (ezt néha *minta-dimenziónak* is nevezik). Az MNIST példában a „minták” számjegyek képei.

Ezenkívül a mély tanulási modellek nem dolgoznak fel egyszerre egy teljes adatkészletet; hanem kis kötegekre bontják az adatokat. Konkrétan, íme egy köteg MNIST számjegyeinkből, 128-as kötegmérettel:
```
batch = train_images[:128]
```
És itt a következő adag:
```
batch = train_images[128:256]
```
És az *n*-edik adag:
```
n = 3
batch = train_images[128 * n:128 * (n + 1)]
```
Ha ilyen kötegtenzorra gondolunk, akkor az első tengelyt (0. tengely) *kötegtengelynek* vagy *kötegdimenziónak* nevezzük. Ez egy olyan kifejezés, amellyel gyakran találkozik a Keras és más mély tanulási könyvtárak használatakor.

### 2.2.8 Valós példák adattenzorokra

Tegyük konkrétabbá az adattenzorokat néhány példával, amelyek hasonlóak ahhoz, amivel később találkozni fog. A kezelni kívánt adatok szinte mindig a következő kategóriák valamelyikébe tartoznak:
* *Vektor adatok* – másodrendű alaktenzorok `(minták, jellemzők)`, ahol minden minta numerikus attribútumok („jellemzők”) vektora
* *Idősor adatok vagy szekvencia adatok* – harmaddendű alakzattenzorok `(minták, időlépések, jellemzők)`, ahol minden minta jellemzővektorok (`időlépés` hosszúságú) sorozata
* *Képek* – alakzatokból álló negyedrendű tenzorok `(minták, magasság, szélesség, csatornák)`, ahol minden minta pixelekből álló 2D rács, és minden pixelt egy értékvektor („csatornák”) képvisel.
* *Videó* – alakzatokból álló ötödrendű tenzorok `(minták, keretek, magasság, szélesség, csatornák)`, ahol minden minta képek (képkocka periódusú) sorozata

### 2.2.9 Vektor adatok

Ez az egyik leggyakoribb eset. Egy ilyen adatkészletben minden egyes adatpont vektorként kódolható, és így egy adatköteg másodrendű tenzorként (vagyis vektortömbként) lesz kódolva, ahol az első tengely a *minta tengelye* és a második tengely a *jellemzők tengelye*. {36.o:}

Nézzünk két példát:
* Emberek biztosítás-matematikai adatkészlete, ahol figyelembe vesszük az egyes személyek korát, nemét és jövedelmét. Minden személy 3 értékből álló vektorként jellemezhető, és így egy teljes, 100 000 emberből álló adathalmaz tárolható egy másodrendű `(100000, 3)` méretű tenzorban.
* Szöveges dokumentumok adathalmaza, ahol az egyes dokumentumokat azzal ábrázoljuk, hogy az egyes szavak hányszor szerepelnek benne (egy 20 000 gyakori szót tartalmazó szótárból). Minden dokumentum 20000 értékből álló vektorként kódolható (a szótárban szavanként egy szám), és így egy 500 dokumentumból álló teljes adatkészlet tárolható egy (500, 20000) méretű tenzorban.

### 2.2.10 Idősor adatok vagy sorrendi adatok

Amikor az idő számít az adatokban (az időrendi sorrend értelmében), akkor célszerű egy explicit időtengellyel rendelkező harmadrendű tenzorban tárolni. Minden minta vektorok sorozataként (másodrendű tenzorként) kódolható, és így egy köteg adatot harmadrendű tenzorként kódolunk (lásd a 2.3. ábrát).

<img src="figs/f2.3_.jpg">

**2.3. ábra** Egy harmadrendű idősor adattenzor

Az időtengely megegyezés szerint mindig a második tengely (az 1-es index tengelye). Nézzünk néhány példát:
* A részvényárfolyamok adatkészlete. Percenként tároljuk a részvény aktuális árfolyamát, az elmúlt perc legnagyobb árfolyamát és az elmúlt perc legkisebb árát. Így minden perc 3D-s vektorként, egy egész kereskedési nap `(390, 3)` méretű mátrixként van kódolva (egy kereskedési napon 390 perc van), és 250 napnyi adat tárolható egy harmadrendű `(250, 390, 3)` méretű tenzorban. Itt minden minta egy nap adata lenne.
* A tweetek adatkészlete, ahol minden egyes tweetet 280 karakterből álló sorozatként kódolunk a 128 egyedi karakterből álló ábécéből. Ebben a beállításban minden karakter 128-as méretű bináris vektorként kódolható (csupa nulla vektor, kivéve a karakternek megfelelő indexnél egy 1-es bejegyzést). Ezután minden tweet kódolható másodrendű `(280, 128)` méretű tenzorként, és egy 1 millió tweetből álló adatkészlet tárolható egy `(1000000, 280, 128)` méretű tenzorban.

### 2.2.11 Képadatok

A képek általában három dimenzióval rendelkeznek: magasság, szélesség és színmélység. Bár a szürkeárnyalatos képeknek (mint az MNIST számjegyeink) csak egyetlen színcsatornájuk van, és így másodrendű tenzorokban tárolhatók, a képtenzorok a megegyezés szerint mindig harmadrendűek, a szürkeárnyalatos képekhez egydimenziós színcsatornával. Így egy 128 db 256 × 256 méretű szürkeárnyalatos képből álló köteg egy `(128, 256, 256, 1)` méretű tenzorban, egy 128 színes képből álló köteg pedig egy `(128, 256, 256, 3)` tenzorban tárolható (lásd a 2.4. ábrát).

<img src="figs/f2.3_.jpg">

**2.4. ábra:** Negyedrendű képadattenzor

Két konvenció létezik a képtenzorok alakjára: a *channels-last* konvenció (amely a TensorFlow-ban szabványos) és a *channels-first* konvenció (amely egyre inkább kegyvesztett).
A channels-last konvenció a színmélység tengelyt a végére helyezi: `(minták, magasság, szélesség, színmélység)`. Eközben a channels-first konvenció a színmélység tengelyét közvetlenül a kötegtengely után helyezi el: `(minták, színmélység, magasság, szélesség)`. A channels-first konvencióval az előző példák `(128, 1, 256, 256)` és `(128, 3, 256, 256)` lesznek. A Keras API mindkét formátumot támogatja.

### 2.2.12 Videó adatok

A videoadatok azon kevés valós-életbeli adattípusok egyike, amelyekhez ötödrendű tenzorokra lesz szükség. A videó felfogható képkockák sorozataként, amelyek mindegyike színes kép. Mivel minden képkocka tárolható harmadrendű tenzorban `(magasság, szélesség, színmélység)`, a képkockák sorozata egy negyedrendű tenzorban tárolható `(képkockák, magasság, szélesség, színmélység)`, és így egy köteg különböző videó ötödrendű tenzorban tárolható `(minták, képkockák, magasság, szélesség, színmélység)`.

Például egy 60 másodperces, 144 × 256 méretű YouTube-videó, amelyet másodpercenként 4 képkocka sebességgel vettek fel, 240 képkockát tartalmazna. Négy ilyen videoklipből álló köteg egy (4, 240, 144, 256, 3) méretű tenzorban kerül tárolásra. Ez összesen 106 168 320 érték! Ha a tenzor `dtype` értéke `float32`, akkor minden érték 32 biten lenne tárolva, így a tenzor 405 MB-ot jelentene. Súlyos! A valós életben talált videók sokkal könnyebbek, mert nem `float32`-ben tárolódnak, és általában nagy tényezővel tömörítik őket (például MPEG formátumban).

## 2.3 A neurális hálózatok fogaskerekei: Tenzorműveletek

Ahogyan bármely számítógépes program végső soron bináris bemeneteken végrehajtott bináris műveletek kis halmazára redukálható (ÉS, VAGY, NOR és így tovább), a mély neurális hálózatok által megtanult összes transzformáció egy maroknyi *tenzorműveletre* (vagy *tenzor függvényre*) redukálható, amiket numerikus adatok tenzoraira alkalmazunk. Például lehetőség van tenzorok összeadására, tenzorok szorzására és így tovább.

Kezdeti példánkban a modellünket úgy építettük fel, hogy `Dense` (sűrű) rétegeket raktunk egymásra. A Keras réteg példánya így néz ki:
```
keras.layers.Dense(512, activation="relu")
```
Ez a réteg függvényként értelmezhető, amely bemenetként egy mátrixot vesz, és egy másik mátrixot ad vissza – a bemeneti tenzor új reprezentációját. Pontosabban, a függvény a következő (ahol W egy mátrix és b egy vektor, mindkét réteg attribútuma):
```
output = relu(dot(input, W) + b)
```
Csomagoljuk ki ezt. Itt három tenzorműveletünk van:
* Pontszorzat (`dot`) a bemeneti tenzor és a W nevű tenzor között
* Összeadás (+) a kapott mátrix és egy b vektor között
* Egy `relu` művelet: `relu(x)` az `max(x, 0)`; A „relu” a „rektifikált lineáris egység” rövidítése.

MEGJEGYZÉS

> Bár ez a rész teljes egészében a lineáris algebrai kifejezésekkel foglalkozik, itt nem talál matematikai jelöléseket. Azt tapasztaltam, hogy a matematikai fogalmakat könnyebben el tudják sajátítani a matematikai háttérrel nem rendelkező programozók, ha matematikai egyenletek helyett rövid Python-részletként fejezik ki őket. Tehát mindvégig a NumPy és a TensorFlow kódot fogjuk használni.

### 2.3.1 Elemenkénti műveletek

A `relu` művelet és az összeadás elemenkénti műveletek: olyan műveletek, amelyek függetlenül kerülnek alkalmazásra a vizsgált tenzorok minden egyes bejegyzésére. Ez azt jelenti, hogy ezek a műveletek nagymértékben alkalmasak masszívan párhuzamos megvalósításokra (*vektorizált* implementációk, ez a kifejezés az 1970–90-es időszak *vektorprocesszoros* szuperszámítógép-architektúrájából származik). Ha naiv Python-megvalósítást szeretne írni egy elemenkénti művelethez, akkor `for` ciklust használjon, mint az elemenkénti `relu` művelet naiv megvalósításában:


In [ ]:
def naive_relu(x):
    assert len(x.shape) == 2      #<--- x egy másodrendű NumPy tenzor.
    x = x.copy()                  #<--- Kerüljük a bemeneti tenzor felülírását.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

Ugyanezt megteheti az összeadáshoz:

In [ ]:
def naive_add(x, y):
    assert len(x.shape) == 2      #<--- x és y másodrendű NumPy tenzor.
    assert x.shape == y.shape
    x = x.copy()                  #<--- Kerüljük a bemeneti tenzor felülírását.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

Ugyanezen az elven elvégezheti az elemenkénti szorzást, kivonást és így tovább.
A gyakorlatban a NumPy tömbök kezelésekor ezek a műveletek jól optimalizált beépített NumPy függvényekként érhetők el, amelyek maguk delegálják a nehéz terheket egy Basic Linear Algebra Subprograms (BLAS) megvalósításra. A BLAS alacsony szintű, nagyon párhuzamos, hatékony tenzormanipulációs rutinok, amelyeket jellemzően Fortran vagy C nyelven valósítanak meg.
Tehát a NumPy-ben képes elvégezni a következő elemenkénti műveletet, és gyorsan fog működni:

In [ ]:
import numpy as np
z = x + y               #<--- elemszintű összeadás
z = np.maximum(z, 0.)   #<--- elemszintű relu

Állapitsuk meg a tényleges időkülönbséget:

In [ ]:
import time

x = np.random.random((20, 100))
y = np.random.random((20, 100))

t0 = time.time()
for _ in range(1000):
    z = x + y
    z = np.maximum(z, 0.)
print("Took: {0:.2f} s".format(time.time() - t0))

Ez csak 0,02 másodpercet vesz igénybe. Miközben a naiv verzió meglepő 2,45 másodpercet kíván:

In [ ]:
t0 = time.time()
for _ in range(1000):
    z = naive_add(x, y)
    z = naive_relu(z)
print("Took: {0:.2f} s".format(time.time() - t0))

Hasonlóképpen, ha TensorFlow kódot futtat egy GPU-n, az elemenkénti műveletek végrehajtása teljesen vektorizált CUDA-megvalósításokon keresztül történik, amelyek a legjobban tudják kihasználni a rendkívül párhuzamos GPU-chip architektúrát:

### 2.3.2 Kiterjesztés (broadcasting)

A `naive_add` korábbi naiv megvalósítása csak az azonos alakú másodrendű tenzorok összeadását támogatja. De a korábban bemutatott `Dense` rétegben összeadtunk egy másodrendű tenzort egy vektorral. Mi történik az összeadással, ha a két összeadandó tenzor alakja különbözik?

Ha lehetséges, és ha nincs kétértelműség, a kisebb tenzort kiterjesztik, hogy megfeleljen a nagyobb tenzor alakjának. A kiterjesztés két lépésből áll:
1. Tengelyek (úgynevezett *broadcast tengelyek*) adódnak hozzá a kisebb tenzorhoz, hogy megfeleljenek a nagyobb tenzor `ndim`-jének.
2. A kisebb tenzor megismétlődik ezen új tengelyek mentén, hogy megfeleljen a nagyobb tenzor teljes alakjának.

Nézzünk egy konkrét példát. Tekintsük a `(32, 10)` méretű `X`-et és a `(10,)` méretű `y`-t:

In [ ]:
import numpy as np
X = np.random.random((32, 10))  #<--- X egy (32, 10) méretű véletlenszerű mátrix.
y = np.random.random((10,))     #<--- y egy (10,) méretű véletlenszerű vektor.

Először egy üres első tengelyt adunk az `y`-hoz, amelynek mérete így `(1, 10)`-re változik:

In [ ]:
y = np.expand_dims(y, axis=0)   #<--- Az y mérete most (1, 10).

Ezután `y`-t 32-szer megismételjük az új tengely mentén, így a `(32, 10)` méretű `Y` tenzort kapjuk, ahol `Y[i, :] == y for i in range(0, 32)`:

In [ ]:
Y = np.concatenate([y] * 32, axis=0)    #<--- Megismételjük az y-t 32-szer a 0 tengely mentén,
                                        #     hogy megkapjuk Y-t, amelynek mérete (32, 10).

Ezen a ponton folytathatjuk az X és Y összeadását, mert azonos alakúak.

A megvalósítás szempontjából nem jön létre új másodrendű tenzor, mert az rettenetesen pazarló lenne. Az ismétlési művelet teljesen virtuális: algoritmikus szinten történik, nem pedig memória szinten. De ha azt gondoljuk, hogy a vektor 10-szer ismétlődik egy új tengely mellett, az hasznos mentális modell. Így nézne ki egy naiv megvalósítás:

In [ ]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2            #<--- x egy másodrendű NumPy tenzor.
    assert len(y.shape) == 1            #<--- y egy NumPy vektor.
    assert x.shape[1] == y.shape[0]
    x = x.copy()                        #<--- Kerüljük a bemeneti tenzor felülírását.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
    return x

A kiterjesztéssel általában elemenkénti műveletek hajthatók végre, amelyek két bemeneti tenzort vesznek át, ha az egyik tenzor mérete `(a, b, … n, n + 1, … m)`, és a másik mérete pedig `(n, n + 1, … m)`. A kiterjesztés ezután automatikusan megtörténik az `a`-tól `n-1` tengelyeken.

A következő példa az elemenkénti `maximum` műveletet két különböző alakú/méretű tenzorra alkalmazza kiterjesztésen/sugárzáson keresztül:

In [ ]:
import numpy as np
x = np.random.random((64, 3, 32, 10))   #<--- x egy véletlenszerű tenzor, amelynek mérete (64, 3, 32, 10).
y = np.random.random((32, 10))          #<--- x egy véletlenszerű tenzor, amelynek mérete (32, 10).
z = np.maximum(x, y)                    #<--- A z kimenet mérete (64, 3, 32, 10), mint az x-é.

### 2.3.3 Tenzor szorzat

A *tenzorszorzat* vagy *pontszorzat* (nem tévesztendő össze az elemi szorzattal, a * operátorral) az egyik leggyakoribb, leghasznosabb tenzorművelet.

A NumPy-ban a tenzorszorzat az `np.dot` függvény segítségével történik (mivel a tenzorszorzat matematikai jelölése általában pont):

In [ ]:
x = np.random.random((32,))
y = np.random.random((32,))
z = np.dot(x, y)

A matematikai felírásban a műveletet egy ponttal (•) jelöljük:
```
z = x • y
```

Matematikailag mit csinál a pontművelet? Kezdjük két vektor, x és y pontszorzatával. A következőképpen kerül kiszámításra:

In [ ]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1    #<--- x és y NumPy vektor
    assert len(y.shape) == 1    #
    assert x.shape[0] == y.shape[0]
    z = 0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z

Biztosan észrevette, hogy a két vektor közötti pontszorzat skalár, és csak az azonos elemszámú vektorok kompatibilisek a pontszorzattal.

használhatja az x mátrix és egy y vektor közötti pontszorzatot is, amely egy olyan vektort ad vissza, ahol az együtthatók az y és az x sorai közötti pontszorzatok. A következőképpen valósithatjuk meg:

Helyesírás matyar?